In [2]:
#Import Required Libraries
import pandas as pd
import numpy as np
import seaborn as sns

#Read the data file
test_reg_df = pd.read_excel('Regression_Data.xlsx',converters={'FDOS':str,'LDOS':str, 'LINE_NUMBER':str})
test_reg_df.head(10)




,ICN_Number,Claim_type,FDOS,LDOS,Charge,Modifier,Type_of_Service,Place_of_Service,Payee_Code,Status,Line_Number,Type_of_Bill,Limit_Type
0,26200350154800,12,120619,120619,17500.0,NaN,C,3,J,A,10.0,111,FDD
1,26200350147500,82,090519,090519,12500.0,NaN,D,3,J,7,10.0,111,FDD
2,20200350083600,13,110219,110819,459200.0,NaN,R,1,J,5,10.0,111,PRS
3,20200350083600,13,110219,110819,158400.0,NaN,S,1,J,5,20.0,111,PRS
4,20200350083600,13,110219,110819,115100.0,NaN,S,1,J,5,30.0,111,PRS
5,20200350083600,13,110219,110819,38400.0,NaN,S,1,J,5,40.0,111,PRS
6,20200350083600,13,110219,110819,28846.0,NaN,S,1,J,5,50.0,111,PRS
7,20200350083600,13,110219,110819,19200.0,NaN,S,1,J,5,60.0,111,PRS
8,20200350083600,13,110219,110819,5339.0,NaN,S,1,J,5,70.0,111,PRS
9,20200350093000,83,120619,121219,750000.0,NaN,R,1,J,7,10.0,111,CCG


In [8]:

#Covert the Dat fields to Date time format.  Calculate the # of days, Split the Program code and claim type
test_reg_df['FDOS']= pd.to_datetime(test_reg_df['FDOS'])
test_reg_df['LDOS']= pd.to_datetime(test_reg_df['LDOS'])
test_reg_df['DOS_Num'] = (test_reg_df['LDOS'] - test_reg_df['FDOS']).dt.days + 1.0
test_reg_df['Program_Code']= test_reg_df['Claim_type'].astype(str).str[0]
test_reg_df['Claim_Class']= test_reg_df['Claim_type'].astype(str).str[1]


conditions = [
    (test_reg_df['Charge']/100 == 0),
    (test_reg_df['Charge']/100 > 0) & (test_reg_df['Charge']/100 < 101),
    (test_reg_df['Charge']/100 > 100) & (test_reg_df['Charge']/100 < 501),
    (test_reg_df['Charge']/100 > 500) & (test_reg_df['Charge']/100 < 1001),
    (test_reg_df['Charge']/100 > 1000) & (test_reg_df['Charge']/100 < 5001),
    (test_reg_df['Charge']/100 > 5000) & (test_reg_df['Charge']/100 < 10001),
    (test_reg_df['Charge']/100 > 10000)]
choices = ['Zero', 'Less than 100', 'Between 100 and 500', 'Between 500 and 1000', 'Between 1000 and 5000', 'Between 5000 and 10000','Greater than 10000']

test_reg_df['Charge_Band'] = np.select(conditions, choices, default='None of the Above')

#Create Program Code Dataframe
Program_Code_df = pd.DataFrame.from_dict({'PC': ['0','1','2','3','4','5','6','7','8','9'],
                   'Desc': ['Non-enrolled Program','Local Program','Federal Employee Program (FEP)',
                            'Central Certification','Inter-plan Bank (IPB) and Reciprocity',
                            'National Program (Non-query)','Inter-plan Teleprocessing Services (ITS)',
                            'Health Maintenance Organization (HMO)','Preferred Provider Organization (PPO)',
                            'Assumed Eligibility Non-FEP']})

#Create Program Code Dataframe
Program_Code_df = pd.DataFrame.from_dict({'PC': ['0','1','2','3','4','5','6','7','8','9'],
                   'Desc': ['Non-enrolled Program','Local Program','Federal Employee Program (FEP)',
                            'Central Certification','Inter-plan Bank (IPB) and Reciprocity',
                            'National Program (Non-query)','Inter-plan Teleprocessing Services (ITS)',
                            'Health Maintenance Organization (HMO)','Preferred Provider Organization (PPO)',
                            'Assumed Eligibility Non-FEP']})

#Create Claim Class Dataframe
Claim_Class_df = pd.DataFrame.from_dict({'CCCode':['1','2','3','4','5','6','7','8','9'],
                                         'Description': ['Hearing','Professional','Facility','Vision','Dental',
                                                         'Prescription','Medicare Facility','Medicare Professional',
                                                         'Advance Pay Medicare Professional']})

#Create Payee Code Dataframe
Payee_Code_df = pd.DataFrame.from_dict({'PCCode':['D','J','M','N','S'],
                                         'Description': ['Unknown (D)',
                                                         'Pay Provider - Address from Provider File (J)',
                                                         'Pay Subscriber - Address from Provider File (M)',
                                                         'Pay Provider - Address from Claim Form (N)',
                                                         'Pay Subscriber - Address from Claim Form (S)']})

#Create Place of Service Dataframe
pos_df =pd.DataFrame.from_dict({'code':['1','2','3','4','5','6','7','8','9','0','A','B','C','D','E','F','J','K','N','P','Y'],
         'Description':['Hospital inpatient','Doctor\'s office','Patient\'s home','Psychiatric day care facility',
                        'Psychiatric night care facility','Hospital outpatient','Nursing home',
                        'Skilled Nursing Facility (SNF)','Ambulance','Other location','Independent clinical laboratory',
                        'Ambulatory surgical center','Residential treatment center','Specialized treatment center',
                        'Comprehensive outpatient rehabilitation center','Independent kidney disease treatment center',
                        'Residential substance abuse facility','Outpatient substance abuse facility',
                        'Adult foster care facility','Pharmacy (Not a HCPCS code)','Hospice']})


#Create Type of Service Dataframe
tos_df =pd.DataFrame.from_dict({'code':['0','1','2','3','4','5','6','7','8','9','A','B','C','D','E','F','G','H','I','J','K','L','M','N','P','R','S','T','U','W','X','Y','Z'],
                               'description':['Blood','Medical Care','Surgery','Consultation','Diagnostic Radiology',
                                              'Laboratory and Pathology','Radiation Therapy','Anesthesia',
                                              'Technical Surgical Assistance','Other Medical Service',
                                              'Used Durable Medical Equipment','Ambulance','Psychiatric',
                                              'Physical Therapy','Maternity','Ambulatory Surgical Center',
                                              'Physical Accessories (purchase)','Hospice','Dental',
                                              'Physical Accessories (rental)','Technical Component',
                                              'Renal Supplies in Home','ESRD Monthly Capitation','Donor Services',
                                              'Professional Component','Accommodations','Ancillaries','Drug',
                                              'Nurses Services','Hearing','Vision','Second Opinion','Third Opinion']})

#Create Charges Dataframe
Charges_df = pd.DataFrame.from_dict({'Band': choices})

#Create Claim level dataframe
Claim_level_df = test_reg_df[['ICN_Number','Program_Code', 'Claim_Class','Payee_Code']]
Claim_level_df = Claim_level_df.drop_duplicates()
Claim_level_df.reset_index()

#Create Type of Service and Place of Service dataframe
pos_tos_df = test_reg_df[['ICN_Number','Type_of_Service', 'Place_of_Service']]
pos_tos_df=pos_tos_df.dropna()
pos_tos_df.info()



<class 'pandas.core.frame.DataFrame'>
Int64Index: 498 entries, 0 to 500
Data columns (total 3 columns):
ICN_Number          498 non-null int64
Type_of_Service     498 non-null object
Place_of_Service    498 non-null object
dtypes: int64(1), object(2)
memory usage: 15.6+ KB


In [12]:
#Create Datafram for DOS
DOS_df = test_reg_df[['ICN_Number','DOS_Num']]
DOS_df = DOS_df.dropna()
DOS_df


,ICN_Number,DOS_Num
0,26200350154800,1.0
1,26200350147500,1.0
2,20200350083600,7.0
3,20200350083600,7.0
4,20200350083600,7.0
5,20200350083600,7.0
6,20200350083600,7.0
7,20200350083600,7.0
8,20200350083600,7.0
9,20200350093000,7.0


In [13]:

#Count the claim class and calculate Percentage
f, u = pd.factorize(Claim_level_df.Claim_Class)
d = dict(zip(u, np.bincount(f)))
Claim_Class_df['count'] = Claim_Class_df.CCCode.map(d)
Claim_Class_df['count']=Claim_Class_df['count'].fillna(0)
Claim_Class_df['percentage']=round(Claim_Class_df['count']/Claim_Class_df['count'].sum()*100,2)

#Create a histogram for Claim class percentage
sns.set(rc={'figure.figsize':(10,10)})
sns.set(style="whitegrid", color_codes=True)
CCChart = sns.barplot(x='Description', y='percentage', data=Claim_Class_df)
CCChart.set(ylabel='Percent')
CCChart.set(xlabel='Claim Class')
CCChart.set_xticklabels(CCChart.get_xticklabels(), 
    rotation=45, 
    horizontalalignment='right');
for p in CCChart.patches:
  CCChart.annotate(format(p.get_height(), '.2f'), 
              (p.get_x() + p.get_width() / 2., 
               p.get_height()), ha = 'center', 
              va = 'center', 
              xytext = (0, 10), 
              textcoords = 'offset points')

CCChart.figure.savefig("Test_Claim_Class_Graph.jpg",dpi=300, bbox_inches='tight')
CCChart.figure.clf()


<Figure size 720x720 with 0 Axes>

In [ ]:

#Count the program code and calculate Percentage
f, u = pd.factorize(Claim_level_df.Program_Code)
d = dict(zip(u, np.bincount(f)))
Program_Code_df['count'] = Program_Code_df.PC.map(d)
Program_Code_df['count']=Program_Code_df['count'].fillna(0)
Program_Code_df['percentage']=round(Program_Code_df['count']/Program_Code_df['count'].sum()*100,2)
Program_Code_df

#Create a histogram for Program code percentage
sns.set(rc={'figure.figsize':(10,10)})
sns.set(style="whitegrid", color_codes=True)
PCChart = sns.barplot(x='Desc', y='percentage', data=Program_Code_df)
PCChart.set(ylabel='Percent')
PCChart.set(xlabel='Program')
PCChart.set_xticklabels(PCChart.get_xticklabels(), 
    rotation=45, 
    horizontalalignment='right');
for p in PCChart.patches:
  PCChart.annotate(format(p.get_height(), '.2f'), 
              (p.get_x() + p.get_width() / 2., 
               p.get_height()), ha = 'center', 
              va = 'center', 
              xytext = (0, 10), 
              textcoords = 'offset points')
PCChart.figure.savefig("Test_Program_Code_Graph.jpg",dpi=300, bbox_inches='tight')
PCChart.figure.clf()

#Count the Payee code and calculate Percentage
f, u = pd.factorize(Claim_level_df.Payee_Code)
d = dict(zip(u, np.bincount(f)))
Payee_Code_df['count'] = Payee_Code_df.PCCode.map(d)
Payee_Code_df['count']=Payee_Code_df['count'].fillna(0)
Payee_Code_df['percentage']=round(Payee_Code_df['count']/Payee_Code_df['count'].sum()*100,2)
Payee_Code_df

#Create a histogram for Payee code percentage
sns.set(rc={'figure.figsize':(4,6)})
sns.set(style="whitegrid", color_codes=True)
PayCodeChart = sns.barplot(x='Description', y='percentage', data=Payee_Code_df)
PayCodeChart.set(ylabel='Percent')
PayCodeChart.set(xlabel='Payee Code')
PayCodeChart.set_xticklabels(PayCodeChart.get_xticklabels(), 
    rotation=45, 
    horizontalalignment='right');
for p in PayCodeChart.patches:
    PayCodeChart.annotate(format(p.get_height(), '.2f'), 
              (p.get_x() + p.get_width() / 2., 
               p.get_height()), ha = 'center', 
              va = 'center', 
              xytext = (0, 10), 
              textcoords = 'offset points')
PayCodeChart.figure.savefig("Test_Payee_Code_Graph.jpg",dpi=300, bbox_inches='tight')
PayCodeChart.figure.clf()

#Count the Place of Service and calculate Percentage
f, u = pd.factorize(pos_tos_df.Place_of_Service)
d = dict(zip(u, np.bincount(f)))
pos_df['count'] = pos_df.code.map(d)
pos_df['count']=pos_df['count'].fillna(0)
pos_df['percentage']=round(pos_df['count']/pos_df['count'].sum()*100,2)
pos_df

#Create a histogram for Place of Service percentage
sns.set(rc={'figure.figsize':(25,6)})
sns.set(style="whitegrid", color_codes=True)
POS_Chart = sns.barplot(x='Description', y='percentage', data=pos_df)
POS_Chart.set(ylabel='Percent')
POS_Chart.set(xlabel='Place of Service')
POS_Chart.set_xticklabels(POS_Chart.get_xticklabels(), 
    rotation=45, 
    horizontalalignment='right');
for p in POS_Chart.patches:
    POS_Chart.annotate(format(p.get_height(), '.2f'), 
              (p.get_x() + p.get_width() / 2., 
               p.get_height()), ha = 'center', 
              va = 'center', 
              xytext = (0, 10), 
              textcoords = 'offset points')
POS_Chart.figure.savefig("Test_Place_Of_Service_Graph.jpg",dpi=300, bbox_inches='tight')
POS_Chart.figure.clf()

#Count the Type of Service and calculate Percentage
f, u = pd.factorize(pos_tos_df.Type_of_Service)
d = dict(zip(u, np.bincount(f)))
tos_df['count'] = tos_df.code.map(d)
tos_df['count']=tos_df['count'].fillna(0)
tos_df['percentage']=round(tos_df['count']/tos_df['count'].sum()*100,2)
tos_df

#Create a histogram for Type of Service percentage
sns.set(rc={'figure.figsize':(30,15)})
sns.set(style="whitegrid", color_codes=True)
TOS_Chart = sns.barplot(x='description', y='percentage', data=tos_df)
TOS_Chart.set(ylabel='Percent')
TOS_Chart.set(xlabel='Type of Service')
TOS_Chart.set_xticklabels(TOS_Chart.get_xticklabels(), 
    rotation=45, 
    horizontalalignment='right');
for p in TOS_Chart.patches:
    TOS_Chart.annotate(format(p.get_height(), '.2f'), 
              (p.get_x() + p.get_width() / 2., 
               p.get_height()), ha = 'center', 
              va = 'center', 
              xytext = (0, 10), 
              textcoords = 'offset points')
TOS_Chart.figure.savefig("Test_Type_Of_Service_Graph.jpg",dpi=300, bbox_inches='tight')
TOS_Chart.figure.clf()

#Count the # of days of Service and calculate Percentage
f, u = pd.factorize(DOS_df.DOS_Num)
d = dict(zip(u, np.bincount(f)))
DOS_cons_df = pd.DataFrame(list(d.items()), columns=['# of days','Count'])
DOS_cons_df['percentage']=round(DOS_cons_df['Count']/DOS_cons_df['Count'].sum()*100,2)

#Create a histogram for Type of Service percentage
sns.set(rc={'figure.figsize':(10,8)})
sns.set(style="whitegrid", color_codes=True)
DOS_plot = sns.barplot(x='# of days', y='percentage', data=DOS_cons_df)
DOS_plot.set(ylabel="Percent")
DOS_plot.set(xlabel="# of Days of Service")
for p in DOS_plot.patches:
    DOS_plot.annotate(format(p.get_height(), '.2f'), 
              (p.get_x() + p.get_width() / 2., 
               p.get_height()), ha = 'center', 
              va = 'center', 
              xytext = (0, 10), 
              textcoords = 'offset points')
DOS_plot.figure.savefig("Test_Date_Of_Service_Graph.jpg",dpi=300, bbox_inches='tight')
DOS_plot.figure.clf()

#Count the Charges band and calculate Percentage
f, u = pd.factorize(test_reg_df.Charge_Band)
d = dict(zip(u, np.bincount(f)))
Charges_df['count'] = Charges_df.Band.map(d)
Charges_df['count']=Charges_df['count'].fillna(0)
Charges_df['percentage']=round(Charges_df['count']/Charges_df['count'].sum()*100,2)
Charges_df

#Create a histogram for Charge Band percentage
sns.set(rc={'figure.figsize':(15,15)})
sns.set(style="whitegrid", color_codes=True)
Charge_Chart = sns.barplot(x='Band', y='percentage', data=Charges_df)
Charge_Chart.set(ylabel='Percent')
Charge_Chart.set(xlabel='Charge Amount')
Charge_Chart.set_xticklabels(Charge_Chart.get_xticklabels(), 
    rotation=45, 
    horizontalalignment='right');
for p in Charge_Chart.patches:
    Charge_Chart.annotate(format(p.get_height(), '.2f'), 
              (p.get_x() + p.get_width() / 2., 
               p.get_height()), ha = 'center', 
              va = 'center', 
              xytext = (0, 10), 
              textcoords = 'offset points')
Charge_Chart.figure.savefig("Test_Charge_Band_Graph.jpg",dpi=300, bbox_inches='tight')
Charge_Chart.figure.clf()

#Count the # of lines and calculate Percentage
group = test_reg_df.groupby(['ICN_Number']).Line_Number.nunique().to_frame('Noofline').reset_index().sort_values(by = 'Noofline')
group1 = group.groupby('Noofline').ICN_Number.count().to_frame('count').reset_index()
group1['claims_per'] = round((group1['count']/group1['count'].sum()) * 100,2)

#Create a histogram for # of claim lines percentage
sns.set(rc={'figure.figsize':(15,15)})
sns.set(style="whitegrid", color_codes=True)
Charge_Chart = sns.barplot(x='Noofline', y='claims_per', data=group1)
Charge_Chart.set(ylabel='Percent')
Charge_Chart.set(xlabel='No. of lines')
Charge_Chart.set_xticklabels(Charge_Chart.get_xticklabels(), 
    horizontalalignment='right');
for p in Charge_Chart.patches:
    Charge_Chart.annotate(format(p.get_height(), '.2f'), 
              (p.get_x() + p.get_width() / 2., 
               p.get_height()), ha = 'center', 
              va = 'center', 
              xytext = (0, 10), 
              textcoords = 'offset points')
Charge_Chart.figure.savefig("Test_No._of_lines_Graph.jpg",dpi=300, bbox_inches='tight')
Charge_Chart.figure.clf()
